### RYNEK MIESZKANIOWY - POLSKA, szybka analiza cen
ŹRÓDŁO DANYCH: NBP

###### Opis zmiennych

RYNEK: 'pierwotny', 'wtórny'

CENY: 'ofertowe', 'transakcyjne'

MIASTA: ['Białystok', 'Bydgoszcz', 'Gdańsk', 'Gdynia', 'Katowice', 'Kielce', 'Kraków', 'Lublin', 'Łódź', 'Olsztyn', 'Opole', 'Poznań', 'Rzeszów', 'Szczecin', 'Warszawa', 'Wrocław', 'Zielona Góra', '7 miast', '10 miast', '6 miast bez Warszawy']

(7 miast =  Gdańsk, Gdynia, Łódź, Kraków, Poznań, Szczecin, Wrocław, Warszawa)


###### Wybór zmiennych

In [18]:
# z opcji przedstawionych powyżej
RYNEK = 'pierwotny'
CENY = 'transakcyjne'
MIASTA = ['Warszawa', 'Gdańsk', 'Lublin', '7 miast', '6 miast bez Warszawy']

###### Import i czyszczenie danych

In [19]:
#import bibliotek
import pandas as pd
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

In [20]:
#słownik do importu
imp_dict = {'pierwotny': {'s_name':'Rynek pierwotny', 'row_start': 5, 'cols': {'ofertowe':'A:U', 'transakcyjne':'X:AR'}},
            'wtórny': {'s_name':'Rynek wtórny', 'row_start': 5, 'cols': {'ofertowe':'A:U', 'transakcyjne':'X:AR'}}}

In [31]:
#import danych NBP
SCIEZKA_DANE_NBP = './dane/ceny_mieszkan.xls'
dane_nbp_raw = pd.read_excel(SCIEZKA_DANE_NBP,
                             sheet_name = imp_dict[RYNEK]['s_name'],
                             skiprows = range(imp_dict[RYNEK]['row_start']), header = 1, usecols = imp_dict[RYNEK]['cols'][CENY])
dane = dane_nbp_raw.copy()
dane.columns = [_col.split('.')[0] for _col in dane.columns]
dane = dane.set_index('Kwartał').apply(lambda x: round(x, 0))
dane = dane.reset_index()
dane = dane[['Kwartał'] + MIASTA]
#ustalenie dynamik
dane_dyn = dane.copy().set_index('Kwartał')
dane_dyn = ((dane_dyn/dane_dyn.shift(4)-1).dropna()*100).apply(lambda x: round(x, 2))
dane_dyn.reset_index(inplace = True)
# dane_dyn.tail()

In [32]:
%%javascript
IPython.OutputArea.auto_scroll_threshold = 9999

<IPython.core.display.Javascript object>

###### Ceny za 1 metr kwadratowy

In [33]:
fig = px.line(dane, x = 'Kwartał', y = MIASTA)
fig.update_layout(title='Ceny ' + CENY + ' mieszkań w Polsce, rynek ' + RYNEK,
                   xaxis_title='kwartał',
                   yaxis_title='cena za 1 metr kwadratowy',
                   legend_title_text='Legenda',
                   hovermode="x unified")
fig.show()

###### Dynamika cen za 1 metr kwadratowy

In [24]:
fig = make_subplots(rows=len(MIASTA), cols=1, subplot_titles = MIASTA, shared_xaxes=True, vertical_spacing = 0.03)
for _counter, _miasto in enumerate(MIASTA):
    fig.add_trace(go.Bar(x = dane_dyn['Kwartał'].values,
                         y = dane_dyn[_miasto].values,
                         name = _miasto),
                  row = _counter + 1, col=1)
fig.update_layout(height = 300 * len(MIASTA), showlegend = False,
                  title='Dynamiki cen mieszkań w Polsce (ceny ' + CENY + ', rynek ' + RYNEK + ')')
fig.show()